In [25]:
import matplotlib.pyplot as plt
plt.style.use('ggplot')
import datetime
import sys
import os
import re
from robobrowser import RoboBrowser
import re
import pandas as pd 
import sqlite3
from contextlib import closing
import preprocessing
browser = RoboBrowser(
    parser='html.parser',  # Beautiful Soupで使用するパーサーを指定する。
    # Cookieが使用できないと表示されてログインできない問題を回避するため、
    # 通常のブラウザーのUser-Agent（ここではFirefoxのもの）を使う。
    user_agent='Mozilla/5.0 (Macintosh; Intel Mac OS X 10.10; rv:45.0) Gecko/20100101 Firefox/45.0')

def main():
    df2=pd.DataFrame()

    # 注文履歴のページを開く。
    print('Navigating...', file=sys.stderr)
    browser.open('https://www.amazon.co.jp/%E3%83%90%E3%82%AB%E3%81%AE%E5%A3%81-%E6%96%B0%E6%BD%AE%E6%96%B0%E6%9B%B8-%E9%A4%8A%E8%80%81-%E5%AD%9F%E5%8F%B8/product-reviews/4106100037/ref=cm_cr_dp_d_show_all_btm?ie=UTF8&reviewerType=all_reviews')


    # ページャーをたどる。
    while True:
        df=pd.DataFrame()
#         assert 'Python' in browser.parsed.title.string  # 注文履歴画面が表示されていることを確認する。
        print(browser.parsed.title.string)
        # print(browser.parsed.prettify())
        df=print_order_history()
        
        df2=pd.concat([df2,df],axis=0,ignore_index=True)

        link_to_next = browser.get_link('次へ')  # 「次へ」というテキストを持つリンクを取得する。
        if not link_to_next:
            return(df2)

        print('Following link to next page...', file=sys.stderr)
        browser.follow_link(link_to_next)  # 「次へ」というリンクをたどる。

def print_order_history():
    print(browser.select('#cm_cr-review_list'))

    # ページ内のすべての注文履歴について反復する。ブラウザーの開発者ツールでclass属性の値を確認できる。
    for i,line_item in enumerate(browser.select('#cm_cr-review_list')):
        review_rank=line_item.select('.a-icon-alt')
        review_date=line_item.select('.review-date')
        review_text=line_item.select('.review-text')
#         review_vote=line_item.select('.cr-vote-text')
        
        if i>0:
            review_rank=[x.get_text() for x in review_rank]
            review_rank=[float(re.sub('5つ星のうち','',x)) for x in review_rank]
#             review_vote=[x.get_text() for x in review_vote]
#             review_vote=[float(re.sub('人のお客様がこれが役に立ったと考えています','',x)) for x in review_vote]
            review_text=[x.get_text() for x in review_text]
            
            review_len_text=[len(x) for x in review_text]
            date_formated=[datetime.datetime.strptime(x.get_text(),'%Y年%m月%d日').strftime('%Y-%m-%d') for x in review_date]
            df=pd.DataFrame({
				'date':date_formated,
				'rank':review_rank,
# 				'vote':review_vote,
				'len_text':review_len_text,
                'text':review_text
			})	
            
            return(df)

In [26]:
df3=main()


Navigating...


Amazon.co.jp：カスタマーレビュー: バカの壁 (新潮新書)
[]


In [ ]:
df3

In [ ]:
# df3['date'] =df3['date'].astype()
datas=[row for row in df3.itertuples(name=None)]
print(datas)    

In [ ]:
dbname='review_of_bakawall.db'
with closing(sqlite3.connect(dbname)) as conn:
    c = conn.cursor()

    # executeメソッドでSQL文を実行する
    create_table = '''create table reviews (id int, date, len int, rate float,
                       review text)'''
    c.execute(create_table)

    # SQL文に値をセットする場合は，Pythonのformatメソッドなどは使わずに，
    # セットしたい場所に?を記述し，executeメソッドの第2引数に?に当てはめる値を
#     # タプルで渡す．
#     sql = 'insert into users (id, rate, age,) values (?,?,?,?)'
#     user = (1, 'Taro', 20, 'male')
#     c.execute(sql, user)

    # 一度に複数のSQL文を実行したいときは，タプルのリストを作成した上で
    # executemanyメソッドを実行する
    insert_sql = 'insert into reviews (id, date, len, rate,review) values (?,?,?,?,?)'
    reviews = datas
    c.executemany(insert_sql, reviews)
    conn.commit()

    select_sql = 'select * from reviews'
    for row in c.execute(select_sql):
        
        print(row)

In [31]:
id_=[x for x in range(1,111)]

In [32]:
id_

[1,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 12,
 13,
 14,
 15,
 16,
 17,
 18,
 19,
 20,
 21,
 22,
 23,
 24,
 25,
 26,
 27,
 28,
 29,
 30,
 31,
 32,
 33,
 34,
 35,
 36,
 37,
 38,
 39,
 40,
 41,
 42,
 43,
 44,
 45,
 46,
 47,
 48,
 49,
 50,
 51,
 52,
 53,
 54,
 55,
 56,
 57,
 58,
 59,
 60,
 61,
 62,
 63,
 64,
 65,
 66,
 67,
 68,
 69,
 70,
 71,
 72,
 73,
 74,
 75,
 76,
 77,
 78,
 79,
 80,
 81,
 82,
 83,
 84,
 85,
 86,
 87,
 88,
 89,
 90,
 91,
 92,
 93,
 94,
 95,
 96,
 97,
 98,
 99,
 100,
 101,
 102,
 103,
 104,
 105,
 106,
 107,
 108,
 109,
 110]